# Reasonable Quote-with Sentimental Analysis

blog link : https://jumbled-marmoset-78e.notion.site/reading-and-writing-electronic-text-week7-zoecai-da08eea26ebe4c9ebaac824117127747

In [1]:
# !pip install -U sentence-transformers

In [2]:
import pandas as pd
from tqdm.notebook import tqdm #进度条
from transformers import pipeline
import math

### 1. news headline (csv)

In [3]:
# news headline
df = pd.read_csv('abcnews-date-text.csv')
df['distance'] = ""
df = df.loc[:2000, :]
df

,publish_date,headline_text,sentiment,score,distance
0,20200101,adelaide riverbank catches alight after new ye...,POSITIVE,0.998660,
1,20200101,adelaides 9pm fireworks spark blaze on riverbank,POSITIVE,0.997871,
2,20200101,archaic legislation governing nt women propert...,NEGATIVE,0.997699,
3,20200101,australia welcomed 2020 with fireworks display...,POSITIVE,0.999707,
4,20200101,authorities says atleast 50 structures have be...,NEGATIVE,0.999677,
...,...,...,...,...,...
1996,20200127,coronavirus schools urge students to be checked,NEGATIVE,0.982530,
1997,20200127,coronavirus strengthening spread infectious du...,NEGATIVE,0.906272,
1998,20200127,court battle over climate change could shake the,NEGATIVE,0.981286,
1999,20200127,difference between grammys song record and alb...,POSITIVE,0.998716,


### 2. quotes (json)

choose one randomly

In [4]:
# quotes
import json
import random

with open("quotes.json","r") as f:
    jsonFiles = json.loads(f.read())

chooseQuote = random.choice(jsonFiles)

chosenQuote = str(chooseQuote['Quote'])
chosenAuthor = str(chooseQuote['Author'] )

chosenQuote + '\n' + chosenAuthor

"And what would happen if we never read the classics? There comes a point in life, it seems to me, where you have to decide whether you're a Person of Letters or merely someone who loves books, and I'm beginning to see that the book lovers have more fun.\nNick Hornby,  Housekeeping vs. the Dirt"

### 3. text comparision

In [6]:
from sentence_transformers import SentenceTransformer
from scipy import spatial

def cal_similar(vector1, vector2):
    return 1 - spatial.distance.cosine(vector1, vector2)

model = SentenceTransformer('all-MiniLM-L6-v2')

quote_embed = model.encode(chosenQuote)
quote_sentiment = pipeline('sentiment-analysis')(chosenQuote)[0]

if quote_sentiment['label'] == "NEGATIVE":
    quote_sentiment = 1 - quote_sentiment['score']
else:
    quote_sentiment = quote_sentiment['score']

for idx in tqdm(range(len(df))):
    headline_embed = model.encode(df['headline_text'][idx])
    df['distance'][idx] = 10 * abs(cal_similar(quote_embed, headline_embed))
    if isinstance(df['sentiment'][idx], str):
        sentiment = df['score'][idx] if df['sentiment'][idx] == "POSITIVE" else 1 - df['score'][idx]
    else:
        sentiment = 0.5
    df['distance'][idx] = df['distance'][idx] + abs(sentiment - quote_sentiment)    

df = df.sort_values(by="distance")

df.head()

# for idx in range(3):
#     print("\033[1;7m" + str(df['publish_date'][idx]) + ',' + df['headline_text'][idx] + "\033[0m")
# print(chosenQuote + '\n————' + chosenAuthor)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


  0%|          | 0/2001 [00:00<?, ?it/s]

/var/folders/f6/kwjf_7yj33d5td_0swg_clx00000gn/T/ipykernel_22027/4240246147.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance'][idx] = 10 * abs(cal_similar(quote_embed, headline_embed))
/var/folders/f6/kwjf_7yj33d5td_0swg_clx00000gn/T/ipykernel_22027/4240246147.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance'][idx] = df['distance'][idx] + abs(sentiment - quote_sentiment)


,publish_date,headline_text,sentiment,score,distance
1949,20200126,khmer rouge daughter decades long search photo...,POSITIVE,0.954420,0.017775
1628,20200122,tim winton novel blueback to commence filming ...,POSITIVE,0.986816,0.028147
1897,20200125,mick keelty on the water beat murray darling b...,POSITIVE,0.990792,0.029586
1803,20200124,nick kyrgios mimicks rafael nadals serve,POSITIVE,0.947689,0.031045
699,20200111,premier to take over tourism portfolio after b...,POSITIVE,0.990439,0.034241
